In [ ]:
import numpy as np
from pathlib import Path
import warnings

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
# This is the folder where your function_1, function_2, ... folders live.
# Adjust "initial_data" if your folder has a slightly different name.
DATA_DIR = Path("initial_data")


def suggest_ucb_point(X, y, beta=1.5, n_candidates=10_000, random_state=0):
    """
    Given current data (X, y) for ONE function, suggest the next query point x* using
    a Gaussian Process + Upper Confidence Bound (UCB) heuristic.

    Parameters
    ----------
    X : np.ndarray, shape (N, d)
        The inputs you've already tried for this function.
        N = number of past points, d = input dimension (2, 3, 4, ..., 8 here).
    y : np.ndarray, shape (N,)
        The outputs you observed for those inputs (same order as rows in X).
    beta : float
        Controls exploration vs exploitation in UCB = mean + beta * std.
        - Larger beta = more exploration (try uncertain points).
        - Smaller beta = more exploitation (stay near current best).
    n_candidates : int
        Base number of random candidate points to try in [0, 1]^d.
        We'll possibly increase this for higher dimensions.
    random_state : int
        Seed for the random number generator so results are reproducible.

    Returns
    -------
    best_x : np.ndarray, shape (d,)
        The chosen next input point (in [0, 1]^d) to submit as your query.
    gp : GaussianProcessRegressor
        The fitted GP model, in case you want to inspect predictions later.
    """

    # --------------------------
    # 1. Basic info about X, y
    # --------------------------
    # X has shape (N, d): N = number of rows (past points), d = dimensions
    N, d = X.shape

    # --------------------------
    # 2. Build the GP kernel
    # --------------------------
    # RBF kernel:
    #   - Think of this as encoding the idea that nearby points in input space
    #     should have similar outputs (smoothness).
    #   - length_scale=np.ones(d) means we start by assuming each dimension
    #     has a length scale of 1.0; the GP can adapt this when fitting.
    #
    # WhiteKernel:
    #   - Adds some noise to the diagonal of the covariance matrix.
    #   - This allows the GP to handle noisy observations instead of trying
    #     to pass exactly through every point.
    kernel = 1.0 * RBF(length_scale=np.ones(d)) + WhiteKernel(noise_level=1e-3)

    # --------------------------
    # 3. Create and fit the GP
    # --------------------------
    gp = GaussianProcessRegressor(
        kernel=kernel,
        normalize_y=True,   # Centers/scales y internally, helpful when magnitudes differ
        random_state=random_state,
    )

    # Fit the GP on the current data (this is where it "learns" the function shape)
    gp.fit(X, y)

    # --------------------------
    # 4. Generate candidate points
    # --------------------------
    # We'll sample random points uniformly within [0, 1]^d.
    # For higher dimensions we increase the number of candidates because
    # the space is bigger and more "empty".
    if d <= 4:
        n = n_candidates
    else:
        # Double for d >= 5 to explore a bit more widely
        n = n_candidates * 2

    # modern NumPy RNG – local, reproducible
    rng = np.random.default_rng(random_state)
    # Xcand has shape (n_candidates, d), with each coordinate between 0 and 1
    Xcand = rng.uniform(0.0, 1.0, size=(n_candidates, d))

    # --------------------------
    # 5. GP predictions at candidates
    # --------------------------
    # For each candidate point, we ask the GP for:
    #   - mu: predicted mean (what y value we expect)
    #   - std: predictive standard deviation (how uncertain we are)
    mu, std = gp.predict(Xcand, return_std=True)

    # --------------------------
    # 6. Compute UCB score
    # --------------------------
    # UCB(x) = mu(x) + beta * std(x)
    #   - mu(x): exploitation (high mean is good)
    #   - std(x): exploration (high uncertainty is good)
    # beta trades between them.
    ucb = mu + beta * std

    # --------------------------
    # 7. Choose the best candidate
    # --------------------------
    # Take the candidate with the highest UCB value.
    best_idx = np.argmax(ucb)
    best_x = Xcand[best_idx]  # shape (d,)

    return best_x, gp


### Function 1

In [ ]:
# Load initial data for function_1
X1 = np.load(DATA_DIR / "function_1" / "initial_inputs.npy")
y1 = np.load(DATA_DIR / "function_1" / "initial_outputs.npy")

# Asserting the shapes of the input and output
print("Function 1 shapes:", X1.shape, y1.shape)

# Asserting the range of the output
print("Function 1 y range:", y1.min(), "to", y1.max())

# Suggest next query point using GP + UCB
best_x1, gp1 = suggest_ucb_point(
    X1,
    y1,
    beta=1.5,
    n_candidates=10_000,
    random_state=41,  # just a seed; you can change if you want
)

# Format as "0.xxxxxx-0.yyyyyy" for the portal
query1 = "-".join(f"{v:.6f}" for v in best_x1)
print("Function 1 query to submit:", query1)


Function 1 shapes: (10, 2) (10,)
Function 1 y range: -0.0036060626443634764 to 7.710875114502849e-16
Function 1 query to submit: 0.954151-0.767932


Function 2

In [3]:
X2 = np.load(DATA_DIR / "function_2" / "initial_inputs.npy")
y2 = np.load(DATA_DIR / "function_2" / "initial_outputs.npy")

print("Function 2 shapes:", X2.shape, y2.shape)
print("Function 2 y range:", y2.min(), "to", y2.max())

best_x2, gp2 = suggest_ucb_point(
    X2,
    y2,
    beta=1.5,
    n_candidates=10_000,
    random_state=42,
)

query2 = "-".join(f"{v:.6f}" for v in best_x2)
print("Function 2 query to submit:", query2)


Function 2 shapes: (10, 2) (10,)
Function 2 y range: -0.06562362443733738 to 0.6112052157614438
Function 2 query to submit: 0.773956-0.438878


Function 3

In [4]:
X3 = np.load(DATA_DIR / "function_3" / "initial_inputs.npy")
y3 = np.load(DATA_DIR / "function_3" / "initial_outputs.npy")

print("Function 3 shapes:", X3.shape, y3.shape)
print("Function 3 y range:", y3.min(), "to", y3.max())

best_x3, gp3 = suggest_ucb_point(
    X3,
    y3,
    beta=1.5,
    n_candidates=10_000,
    random_state=43,
)

query3 = "-".join(f"{v:.6f}" for v in best_x3)
print("Function 3 query to submit:", query3)


Function 3 shapes: (15, 3) (15,)
Function 3 y range: -0.3989255131463011 to -0.034835313350078584
Function 3 query to submit: 0.652299-0.043775-0.020030


Function 4

In [5]:
X4 = np.load(DATA_DIR / "function_4" / "initial_inputs.npy")
y4 = np.load(DATA_DIR / "function_4" / "initial_outputs.npy")

print("Function 4 shapes:", X4.shape, y4.shape)
print("Function 4 y range:", y4.min(), "to", y4.max())

best_x4, gp4 = suggest_ucb_point(
    X4,
    y4,
    beta=1.5,
    n_candidates=10_000,
    random_state=44,
)

query4 = "-".join(f"{v:.6f}" for v in best_x4)
print("Function 4 query to submit:", query4)


Function 4 shapes: (30, 4) (30,)
Function 4 y range: -32.625660215962455 to -4.025542281908162
Function 4 query to submit: 0.372882-0.447822-0.347304-0.451380


Function 5

In [6]:
X5 = np.load(DATA_DIR / "function_5" / "initial_inputs.npy")
y5 = np.load(DATA_DIR / "function_5" / "initial_outputs.npy")

print("Function 5 shapes:", X5.shape, y5.shape)
print("Function 5 y range:", y5.min(), "to", y5.max())

best_x5, gp5 = suggest_ucb_point(
    X5,
    y5,
    beta=1.5,
    n_candidates=10_000,
    random_state=45,
)

query5 = "-".join(f"{v:.6f}" for v in best_x5)
print("Function 5 query to submit:", query5)


Function 5 shapes: (20, 4) (20,)
Function 5 y range: 0.1129397953712203 to 1088.8596181962705
Function 5 query to submit: 0.573131-0.528491-0.763650-0.811693


Function 6

In [7]:
X6 = np.load(DATA_DIR / "function_6" / "initial_inputs.npy")
y6 = np.load(DATA_DIR / "function_6" / "initial_outputs.npy")

print("Function 6 shapes:", X6.shape, y6.shape)
print("Function 6 y range:", y6.min(), "to", y6.max())

best_x6, gp6 = suggest_ucb_point(
    X6,
    y6,
    beta=1.5,
    n_candidates=10_000,
    random_state=46,
)

query6 = "-".join(f"{v:.6f}" for v in best_x6)
print("Function 6 query to submit:", query6)


Function 6 shapes: (20, 5) (20,)
Function 6 y range: -2.5711696316081234 to -0.7142649478202404
Function 6 query to submit: 0.017255-0.359724-0.834147-0.516449-0.535116


Function 7

In [8]:
X7 = np.load(DATA_DIR / "function_7" / "initial_inputs.npy")
y7 = np.load(DATA_DIR / "function_7" / "initial_outputs.npy")

print("Function 7 shapes:", X7.shape, y7.shape)
print("Function 7 y range:", y7.min(), "to", y7.max())

best_x7, gp7 = suggest_ucb_point(
    X7,
    y7,
    beta=1.5,
    n_candidates=10_000,
    random_state=47,
)

query7 = "-".join(f"{v:.6f}" for v in best_x7)
print("Function 7 query to submit:", query7)


Function 7 shapes: (30, 6) (30,)
Function 7 y range: 0.0027014650245082332 to 1.3649683044991994
Function 7 query to submit: 0.741802-0.753669-0.465181-0.103725-0.966852-0.320750


Function 8

In [9]:
X8 = np.load(DATA_DIR / "function_8" / "initial_inputs.npy")
y8 = np.load(DATA_DIR / "function_8" / "initial_outputs.npy")

print("Function 8 shapes:", X8.shape, y8.shape)
print("Function 8 y range:", y8.min(), "to", y8.max())

best_x8, gp8 = suggest_ucb_point(
    X8,
    y8,
    beta=1.5,
    n_candidates=10_000,
    random_state=48,
)

query8 = "-".join(f"{v:.6f}" for v in best_x8)
print("Function 8 query to submit:", query8)


Function 8 shapes: (40, 8) (40,)
Function 8 y range: 5.5921933895401965 to 9.598482002566342
Function 8 query to submit: 0.062610-0.096925-0.100610-0.088544-0.998209-0.369043-0.143785-0.738436
